In [1]:
import numpy as np
import pandas as pd
import multiprocessing as mp

from tqdm import tqdm
from index import get_tokens

In [2]:
df = pd.read_hdf('../../data/clean_articles.h5')

In [3]:
df.dtypes

fetch_date        datetime64[ns]
url                       object
title                     object
ttitle_len                 int64
content                   object
tcontent_len               int64
author                    object
published_date    datetime64[ns]
publisher                 object
estimate_time              int64
likes                      int64
tags                      object
comments                   int64
dtype: object

In [30]:
len(df)

10000

In [28]:
df = df.rename(columns={'content_len': 'tcontent_len'})

In [5]:
tokened_titles_len = []
for s in tqdm(df.title):
    tokened_titles_len.append(len(get_tokens(s)))

100%|██████████| 10000/10000 [00:04<00:00, 2346.03it/s]


In [6]:
df.ttitle_len = tokened_titles_len

In [13]:
tokened_titles_len[:4]

[15, 3, 9, 8]

In [ ]:
content_len = []
for s in tqdm(df.content):
    content_len.append(len(get_tokens(s)))

In [7]:
def worker(s):
    return len(get_tokens(s))

content_len = None
with mp.Pool(processes=16) as pool:
    content_len = pool.map(worker, df.content.tolist())

In [8]:
df.tcontent_len = content_len

In [23]:
df.insert(5, 'content_len', content_len)

In [14]:
df.insert(3, "tokened_titles_len", tokened_titles_len)

In [19]:
df.isnull().values.any()

False

In [20]:
df.head()

,fetch_date,url,title,content,author,published_date,publisher,estimate_time,likes,tags,comments
0,2017-11-23 03:32:02,https://medium.com/@cnizzardini/linux-search-f...,Linux Search From Shell Using Grep Find Xargs,The find command is used to find files in a Un...,Chris,2007-06-04 18:37:45.000,medium,0,0,linux find grep xargs,0
1,2017-11-23 03:32:02,https://medium.com/nome-do-jogo/adeus-windows-...,Adeus Windows,Comecei minha carreira profissional com o Visu...,Carlos Brando,2007-10-01 00:00:00.000,medium,120,3,windows ubuntu linux,0
2,2017-11-23 03:32:02,https://medium.com/security-thinking-cap/802-1...,802.11n Wireless-N devices,It is not out. Dlink and Netgear have some 802...,Eric Vanderburg,2006-05-02 22:09:57.000,medium,0,0,dlink netgear wifi wireless,0
3,2017-11-23 03:32:03,https://medium.com/@akwhitney/mathochism-the-j...,Mathochism: The joys of MOOCulus,One woman’s attempt to revisit the math that p...,AK Whitney,2016-02-01 22:09:57.203,medium,240,1,education mathematics calculus,0
4,2017-11-23 03:32:03,https://medium.com/@brianmmcguire/meditation-o...,"Meditation on Calculus, Emptiness, and The Res...",Perugino (Pietro di Cristoforo Vannucci) (Ital...,Brian M. McGuire,2016-03-27 19:25:25.005,medium,180,2,easter calculus meditation,0


In [13]:
df.content[df.content.isnull()]

935    NaN
Name: content, dtype: object

In [14]:
df.content = df.content.fillna('')

In [2]:
df = pd.read_csv('../../data/articles.csv')

In [77]:
df.head()

,fetch_date,url,title,content,author,published_date,publisher,estimate_time,likes,tags,comments
0,2017-11-23 03:32:02,https://medium.com/@cnizzardini/linux-search-f...,Linux Search From Shell Using Grep Find Xargs,The find command is used to find files in a Un...,Chris,2007-06-04 18:37:45.000,medium,0,0,linux find grep xargs,0
1,2017-11-23 03:32:02,https://medium.com/nome-do-jogo/adeus-windows-...,Adeus Windows,Comecei minha carreira profissional com o Visu...,Carlos Brando,2007-10-01 00:00:00.000,medium,120,3,windows ubuntu linux,0
2,2017-11-23 03:32:02,https://medium.com/security-thinking-cap/802-1...,802.11n Wireless-N devices,It is not out. Dlink and Netgear have some 802...,Eric Vanderburg,2006-05-02 22:09:57.000,medium,0,0,dlink netgear wifi wireless,0
3,2017-11-23 03:32:03,https://medium.com/@akwhitney/mathochism-the-j...,Mathochism: The joys of MOOCulus,One woman’s attempt to revisit the math that p...,AK Whitney,2016-02-01 22:09:57.203,medium,240,1,education mathematics calculus,0
4,2017-11-23 03:32:03,https://medium.com/@brianmmcguire/meditation-o...,"Meditation on Calculus, Emptiness, and The Res...",Perugino (Pietro di Cristoforo Vannucci) (Ital...,Brian M. McGuire,2016-03-27 19:25:25.005,medium,180,2,easter calculus meditation,0


In [76]:
df.dtypes

fetch_date        datetime64[ns]
url                       object
title                     object
content                   object
author                    object
published_date    datetime64[ns]
publisher                 object
estimate_time              int64
likes                      int64
tags                      object
comments                   int64
dtype: object

In [75]:
column_order = ['fetch_date', 'url', 
                'title', 'content', 
                'author', 'published_date', 'publisher', 'estimate_time', 
                'likes', 'tags', 'comments']
df = df[column_order]
df.sort_values(column_order, inplace=True)
df.reset_index(drop=True, inplace=True)

In [9]:
%time df.to_hdf('../../data/clean_articles.h5', 'df', mode='w', format='t', complevel=9)

CPU times: user 1min 27s, sys: 31.5 s, total: 1min 59s
Wall time: 2min 17s


In [32]:
!du -sh clean_articles.h5

 43M	clean_articles.h5
